# **1. Import Libraries**

In [8]:
import requests
import numpy
import pandas as pd

# **2. Endpoints Research**

## **Players Related Info:**

- <pre><b>CommonAllPlayers</b>             # Get all the player' ids from all seasons
https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/commonallplayers.md
</pre>

- <pre><b>CommonPlayerInfo</b>             # Extra information related to the players
https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/commonplayerinfo.md
</pre>

- <pre><b>PlayerAwards</b>                 # Check correlation between teams and allstar players
https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/playerawards.md
</pre>

- <pre><b>LeagueDashPlayerBioStats</b>     # Check correlation between physical aspects and performance
https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/leaguedashplayerbiostats.md
</pre>

## **Teams Related Info:**

- <pre><b>CommonTeamRoster</b>             # Check each player position
https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/commonteamroster.md
</pre>

- <pre><b>TeamDetails</b>                  # Check team awards
https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/teamdetails.md
</pre>

- <pre><b>CumeStatsTeam</b>                # Explore Raptors stats
https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/cumestatsteam.md
</pre>

## **Games Related Info:**

- <pre><b>BoxScoreSummaryV2</b>            # Check correlation between winning and points per quarter
https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/boxscoresummaryv2.md
</pre>

- CumeStatsTeam -> https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/cumestatsteam.md
- LeagueDashTeamStats -> https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/leaguedashteamstats.md

In [9]:
from nba_api.stats.endpoints import commonallplayers, commonplayerinfo, playerawards, leaguedashplayerbiostats, teamdetails, commonteamroster, boxscoresummaryv2

In [22]:
#all_players_info = commonallplayers.CommonAllPlayers(league_id='00', season='2002-03')
#all_players_df = all_players_info.get_data_frames()[0]
#all_players = all_players_df.PERSON_ID

# **3. Functions**

In [40]:
seasons = [f'{x}-{str(x+1)[2:]}' for x in range(1996,2023)]

In [89]:
def leaguedash_season(season, leaguedash_df):

    try:
        leaguedash_info = leaguedashplayerbiostats.LeagueDashPlayerBioStats(league_id='00', season=season)
        df_append = leaguedash_info.get_data_frames()[0]
        df_append['SEASON'] = season
        leaguedash_df = pd.concat([leaguedash_df, df_append], ignore_index=True)
    except:
        pass
    
    return leaguedash_df

In [90]:
def leaguedash_allseasons(seasons):
    
    leaguedash_df = pd.DataFrame()

    for season in seasons:
        leaguedash_df = leaguedash_season(season, leaguedash_df)
    
    all_seasons = leaguedash_df.SEASON.unique()
    missing = [x for x in seasons if x not in all_seasons]

    while missing:
        leaguedash_df = leaguedash_season(season, leaguedash_df)

        all_seasons = leaguedash_df.SEASON.unique()
        missing = [x for x in seasons if x not in all_seasons]
    
    return leaguedash_df


In [91]:
leaguedash_df = leaguedash_allseasons(seasons)

In [83]:
leaguedash_df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COLLEGE,COUNTRY,...,PTS,REB,AST,NET_RATING,OREB_PCT,DREB_PCT,USG_PCT,TS_PCT,AST_PCT,SEASON
0,920,A.C. Green,1.610613e+09,DAL,33.0,6-9,81.0,225,Oregon State,USA,...,597,656,69,-7.4,0.089,0.184,0.118,0.523,0.045,1996-97
1,243,Aaron McKie,1.610613e+09,DET,24.0,6-5,77.0,209,Temple,USA,...,433,221,161,3.7,0.026,0.113,0.142,0.524,0.163,1996-97
2,1425,Aaron Williams,1.610613e+09,VAN,25.0,6-9,81.0,225,Xavier,USA,...,203,143,15,-9.3,0.113,0.144,0.161,0.599,0.051,1996-97
3,768,Acie Earl,1.610613e+09,MIL,27.0,6-11,83.0,240,Iowa,USA,...,188,96,20,-6.4,0.067,0.122,0.220,0.435,0.077,1996-97
4,228,Adam Keefe,1.610613e+09,UTA,27.0,6-9,81.0,241,Stanford,USA,...,235,216,32,7.2,0.096,0.158,0.124,0.572,0.051,1996-97


In [ ]:
leaguedash_df.shape

(11904, 24)

In [ ]:
len(leaguedash_df.SEASON.unique()) == len(seasons)

array(['1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02',
       '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09',
       '2009-10', '2010-11', '2012-13', '2013-14', '2014-15', '2015-16',
       '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
       '2022-23'], dtype=object)

In [ ]:
player_id = 2544  #Lebron

In [ ]:
player_awards_info = playerawards.PlayerAwards(player_id=player_id)
player_awards_df = player_awards_info.get_data_frames()[0]

In [ ]:
player_awards_df